In [27]:
import open3d as o3d
import pandas as pd
import time
import os

def load_csv(file_path):
    try:
        df = pd.read_csv(file_path, sep=";")
        print(df.shape)
        if df.empty:
            print("The CSV file is empty.")
            return None
        return df
    except pd.errors.EmptyDataError:
        print("No data in the CSV file.")
        return None
    except pd.errors.ParserError:
        print("Error parsing the CSV file.")
        return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

def get_point_cloud(file_path, noise = False):
    df = load_csv(file_path)
    if noise: coord = ['X', 'Y', 'Z']
    else: coord = ['X_noise', 'Y_noise', 'Z_noise']

    points = df[coord].values
    colors = df[['red', 'green', 'blue']].values

    point_cloud = o3d.geometry.PointCloud()
    point_cloud.points = o3d.utility.Vector3dVector(points)
    point_cloud.colors = o3d.utility.Vector3dVector(colors)

    return point_cloud

def visualize_point_cloud(file_path, noise = False):

    pcd = get_point_cloud(file_path, noise)
    
    vis = o3d.visualization.Visualizer()
    vis.create_window(window_name='Virtual Information Space', width=950, height=1080, left=0, top=35)
    vis.add_geometry(pcd)

    last_modified_time = os.path.getmtime(file_path)

    print(time.ctime(last_modified_time))

    try:
        while True:
            current_modified_time = os.path.getmtime(file_path)
            
            if current_modified_time != last_modified_time:
                temp = get_point_cloud(file_path, noise)
                if temp is not None: 
                    pcd = temp
                    print(pcd.points)
                    vis.clear_geometries()
                    vis.add_geometry(pcd)
                last_modified_time = current_modified_time
            
            vis.update_geometry(pcd)
            vis.poll_events()
            vis.update_renderer()

    except KeyboardInterrupt:
        print("Stopped by user")
        
    vis.destroy_window()

file_path = 'C:\\Programming\\Fourth Semester\\PointVIS\\PointVIS\\assets\\output\\chair2_frames_17_to_17.csv'  # Replace with the path to your .csv file
visualize_point_cloud(file_path, noise = False)
print("Hello")

(1059, 14)
Thu Jun 13 01:24:09 2024
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 
(1056, 14)
std::vector<Eigen::Vector3d> with 1056 elements.
Use numpy.asarray() to access data.
(1056, 14)
std::vector<Eigen::Vector3d> with 1056 elements.
Use numpy.asarray() to access data.
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation o